In [1]:
from kafka import KafkaProducer
import json
import json

import zlib
import base64
import json

import asyncio
import json

from signalr_async.net import Hub
from signalr_async.net.client import SignalRClient
from dotenv import load_dotenv
import os 

load_dotenv(dotenv_path='../.env')
SERVER = os.getenv('SERVER')


In [2]:
def json_serializer(data):
    return json.dumps(data).encode("utf-8")

producer = KafkaProducer(bootstrap_servers=[SERVER + ':9092'],
                         value_serializer=json_serializer)

def parse_data(text, zipped=True):
    """Parse json and jsonStream as returned by livetiming.formula1.com

    This function can only pass one data entry at a time, not a whole response.
    Timestamps and data need to be separated before and only the data must be passed as a string to be parsed.

    Args:
        text (str): The string which should be parsed
        zipped (bool): Whether or not the text is compressed. This is the case for '.z' data (e.g. position data=)

    Returns:
        Depending on data of which page is parsed:
            - a dictionary created as a result of loading json data
            - a string
    """
    if text[0] == '{':
        return json.loads(text)
    if text[0] == '"':
        text = text.strip('"')
    if zipped:
        text = zlib.decompress(base64.b64decode(text), -zlib.MAX_WBITS)
        return parse_data(text.decode('utf-8-sig'))
    print("Couldn't parse text")
    return text

In [3]:
class F1SignalRClient(SignalRClient):
    driver_list = {}
    timing_data = {}
    session_data = {}
    session_info = {}

    def SessionData(self, data):
        print('SessionData')
        print(data)
        producer.send("session_data", data)


    def SessionInfo(self, data):
        print('SessionInfo')
        print(data)
        producer.send("session_info", data)


    def DriverList(self, data):
        print('DriverList')
        print(data)
        for driver in data:
            if driver != '_kf':
                producer.send("driver_list", data[driver])


    def TimingData(self, data):
        print('TimingData')
        print(data)
        for key in data['Lines']:
            producer.send("timing_data", {key: data['Lines'][key]})

    # def CarData(self, data):
    #     data = parse_data(data)
    #     print('test')
    #     producer.send("F1", data)
    #     print('test')
    #     # print('CarData')
    #     # merge(self.driver_list, drivers)

    def Position(self, data):
        print('Position')
        data = parse_data(data)
        print(data)
        producer.send("position", data)


    async def _process_message(self, message):
        if hasattr(message, "result"):
            for k in message.result:
                if hasattr(self, k):
                    getattr(self, k)(message.result[k])
                elif hasattr(self, k[:-2]):
                    getattr(self, k[:-2])(message.result[k])
        elif hasattr(message, "arguments"):

            k, v, t = message.arguments
            if hasattr(self, k):
                    getattr(self, k)(v)
            elif hasattr(self, k[:-2]):
                getattr(self, k[:-2])(v)


_connection_url = 'https://livetiming.formula1.com/signalr'
hub = Hub("streaming")

async def run_client():
    async with F1SignalRClient(
            _connection_url,
            [hub],
            keepalive_interval=5,
    ) as client:
        await hub.invoke("Subscribe", ["Heartbeat", "CarData.z", "Position.z",
                       "ExtrapolatedClock", "TopThree", "RcmSeries",
                       "TimingStats", "TimingAppData",
                       "WeatherData", "TrackStatus", "DriverList",
                       "RaceControlMessages", "SessionInfo",
                       "SessionData", "LapCount", "TimingData"])
        await client.wait(timeout=5)



In [4]:
# https://github.com/iebb/PixooLiveTiming

await asyncio.gather(
        run_client()
    )



Position
{'Position': [{'Timestamp': '2023-07-02T14:33:14.3328332Z', 'Entries': {'1': {'Status': 'OnTrack', 'X': 1003, 'Y': -1067, 'Z': 7230}, '2': {'Status': 'OnTrack', 'X': 1421, 'Y': -956, 'Z': 7315}, '4': {'Status': 'OnTrack', 'X': 1527, 'Y': -928, 'Z': 7312}, '10': {'Status': 'OnTrack', 'X': 1717, 'Y': -880, 'Z': 7313}, '11': {'Status': 'OnTrack', 'X': 1095, 'Y': -1043, 'Z': 7313}, '14': {'Status': 'OnTrack', 'X': 1479, 'Y': -940, 'Z': 7315}, '16': {'Status': 'OnTrack', 'X': 1083, 'Y': -1046, 'Z': 7316}, '18': {'Status': 'OnTrack', 'X': 1702, 'Y': -883, 'Z': 7313}, '20': {'Status': 'OnTrack', 'X': 1587, 'Y': -913, 'Z': 7312}, '21': {'Status': 'OnTrack', 'X': 1526, 'Y': -928, 'Z': 7310}, '22': {'Status': 'OnTrack', 'X': 1568, 'Y': -917, 'Z': 7230}, '23': {'Status': 'OnTrack', 'X': 1746, 'Y': -873, 'Z': 7230}, '24': {'Status': 'OnTrack', 'X': 1585, 'Y': -913, 'Z': 7311}, '27': {'Status': 'OnTrack', 'X': -7529, 'Y': 5654, 'Z': 7848}, '31': {'Status': 'OnTrack', 'X': 1554, 'Y': -921, 

CancelledError: 

## Data exploration

In [ ]:
driverlist = {'1': {'RacingNumber': '1', 'BroadcastName': 'M VERSTAPPEN', 'FullName': 'Max VERSTAPPEN', 'Tla': 'VER', 'Line': 1, 'TeamName': 'Red Bull Racing', 'TeamColour': '3671C6', 'FirstName': 'Max', 'LastName': 'Verstappen', 'Reference': 'MAXVER01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/M/MAXVER01_Max_Verstappen/maxver01.png.transform/1col/image.png', 'CountryCode': 'NED'}, '14': {'RacingNumber': '14', 'BroadcastName': 'F ALONSO', 'FullName': 'Fernando ALONSO', 'Tla': 'ALO', 'Line': 2, 'TeamName': 'Aston Martin', 'TeamColour': '358C75', 'FirstName': 'Fernando', 'LastName': 'Alonso', 'Reference': 'FERALO01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/F/FERALO01_Fernando_Alonso/feralo01.png.transform/1col/image.png', 'CountryCode': 'ESP'}, '31': {'RacingNumber': '31', 'BroadcastName': 'E OCON', 'FullName': 'Esteban OCON', 'Tla': 'OCO', 'Line': 3, 'TeamName': 'Alpine', 'TeamColour': '2293D1', 'FirstName': 'Esteban', 'LastName': 'Ocon', 'Reference': 'ESTOCO01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/E/ESTOCO01_Esteban_Ocon/estoco01.png.transform/1col/image.png', 'CountryCode': 'FRA'}, '55': {'RacingNumber': '55', 'BroadcastName': 'C SAINZ', 'FullName': 'Carlos SAINZ', 'Tla': 'SAI', 'Line': 8, 'TeamName': 'Ferrari', 'TeamColour': 'F91536', 'FirstName': 'Carlos', 'LastName': 'Sainz', 'Reference': 'CARSAI01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/C/CARSAI01_Carlos_Sainz/carsai01.png.transform/1col/image.png', 'CountryCode': 'ESP'}, '44': {'RacingNumber': '44', 'BroadcastName': 'L HAMILTON', 'FullName': 'Lewis HAMILTON', 'Tla': 'HAM', 'Line': 4, 'TeamName': 'Mercedes', 'TeamColour': '6CD3BF', 'FirstName': 'Lewis', 'LastName': 'Hamilton', 'Reference': 'LEWHAM01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/L/LEWHAM01_Lewis_Hamilton/lewham01.png.transform/1col/image.png', 'CountryCode': 'GBR'}, '16': {'RacingNumber': '16', 'BroadcastName': 'C LECLERC', 'FullName': 'Charles LECLERC', 'Tla': 'LEC', 'Line': 6, 'TeamName': 'Ferrari', 'TeamColour': 'F91536', 'FirstName': 'Charles', 'LastName': 'Leclerc', 'Reference': 'CHALEC01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/C/CHALEC01_Charles_Leclerc/chalec01.png.transform/1col/image.png', 'CountryCode': 'MON'}, '10': {'RacingNumber': '10', 'BroadcastName': 'P GASLY', 'FullName': 'Pierre GASLY', 'Tla': 'GAS', 'Line': 7, 'TeamName': 'Alpine', 'TeamColour': '2293D1', 'FirstName': 'Pierre', 'LastName': 'Gasly', 'Reference': 'PIEGAS01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/P/PIEGAS01_Pierre_Gasly/piegas01.png.transform/1col/image.png', 'CountryCode': 'FRA'}, '63': {'RacingNumber': '63', 'BroadcastName': 'G RUSSELL', 'FullName': 'George RUSSELL', 'Tla': 'RUS', 'Line': 5, 'TeamName': 'Mercedes', 'TeamColour': '6CD3BF', 'FirstName': 'George', 'LastName': 'Russell', 'Reference': 'GEORUS01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/G/GEORUS01_George_Russell/georus01.png.transform/1col/image.png', 'CountryCode': 'GBR'}, '22': {'RacingNumber': '22', 'BroadcastName': 'Y TSUNODA', 'FullName': 'Yuki TSUNODA', 'Tla': 'TSU', 'Line': 15, 'TeamName': 'AlphaTauri', 'TeamColour': '5E8FAA', 'FirstName': 'Yuki', 'LastName': 'Tsunoda', 'Reference': 'YUKTSU01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/Y/YUKTSU01_Yuki_Tsunoda/yuktsu01.png.transform/1col/image.png', 'CountryCode': 'JPN'}, '4': {'RacingNumber': '4', 'BroadcastName': 'L NORRIS', 'FullName': 'Lando NORRIS', 'Tla': 'NOR', 'Line': 9, 'TeamName': 'McLaren', 'TeamColour': 'F58020', 'FirstName': 'Lando', 'LastName': 'Norris', 'Reference': 'LANNOR01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/L/LANNOR01_Lando_Norris/lannor01.png.transform/1col/image.png', 'CountryCode': 'GBR'}, '81': {'RacingNumber': '81', 'BroadcastName': 'O PIASTRI', 'FullName': 'Oscar PIASTRI', 'Tla': 'PIA', 'Line': 10, 'TeamName': 'McLaren', 'TeamColour': 'F58020', 'FirstName': 'Oscar', 'LastName': 'Piastri', 'Reference': 'OSCPIA01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/O/OSCPIA01_Oscar_Piastri/oscpia01.png.transform/1col/image.png', 'CountryCode': 'AUS'}, '21': {'RacingNumber': '21', 'BroadcastName': 'N DE VRIES', 'FullName': 'Nyck DE VRIES', 'Tla': 'DEV', 'Line': 12, 'TeamName': 'AlphaTauri', 'TeamColour': '5E8FAA', 'FirstName': 'Nyck', 'LastName': 'De Vries', 'Reference': 'NYCDEV01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/N/NYCDEV01_Nyck_De%20Vries/nycdev01.png.transform/1col/image.png', 'CountryCode': 'NED'}, '23': {'RacingNumber': '23', 'BroadcastName': 'A ALBON', 'FullName': 'Alexander ALBON', 'Tla': 'ALB', 'Line': 14, 'TeamName': 'Williams', 'TeamColour': '37BEDD', 'FirstName': 'Alexander', 'LastName': 'Albon', 'Reference': 'ALEALB01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/A/ALEALB01_Alexander_Albon/alealb01.png.transform/1col/image.png', 'CountryCode': 'THA'}, '18': {'RacingNumber': '18', 'BroadcastName': 'L STROLL', 'FullName': 'Lance STROLL', 'Tla': 'STR', 'Line': 20, 'TeamName': 'Aston Martin', 'TeamColour': '358C75', 'FirstName': 'Lance', 'LastName': 'Stroll', 'Reference': 'LANSTR01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/L/LANSTR01_Lance_Stroll/lanstr01.png.transform/1col/image.png', 'CountryCode': 'CAN'}, '77': {'RacingNumber': '77', 'BroadcastName': 'V BOTTAS', 'FullName': 'Valtteri BOTTAS', 'Tla': 'BOT', 'Line': 11, 'TeamName': 'Alfa Romeo', 'TeamColour': 'C92D4B', 'FirstName': 'Valtteri', 'LastName': 'Bottas', 'Reference': 'VALBOT01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/V/VALBOT01_Valtteri_Bottas/valbot01.png.transform/1col/image.png', 'CountryCode': 'FIN'}, '2': {'RacingNumber': '2', 'BroadcastName': 'L SARGEANT', 'FullName': 'Logan SARGEANT', 'Tla': 'SAR', 'Line': 18, 'TeamName': 'Williams', 'TeamColour': '37BEDD', 'FirstName': 'Logan', 'LastName': 'Sargeant', 'Reference': 'LOGSAR01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/L/LOGSAR01_Logan_Sargeant/logsar01.png.transform/1col/image.png', 'CountryCode': 'USA'}, '20': {'RacingNumber': '20', 'BroadcastName': 'K MAGNUSSEN', 'FullName': 'Kevin MAGNUSSEN', 'Tla': 'MAG', 'Line': 19, 'TeamName': 'Haas F1 Team', 'TeamColour': 'B6BABD', 'FirstName': 'Kevin', 'LastName': 'Magnussen', 'Reference': 'KEVMAG01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/K/KEVMAG01_Kevin_Magnussen/kevmag01.png.transform/1col/image.png', 'CountryCode': 'DEN'}, '27': {'RacingNumber': '27', 'BroadcastName': 'N HULKENBERG', 'FullName': 'Nico HULKENBERG', 'Tla': 'HUL', 'Line': 17, 'TeamName': 'Haas F1 Team', 'TeamColour': 'B6BABD', 'FirstName': 'Nico', 'LastName': 'Hulkenberg', 'Reference': 'NICHUL01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/N/NICHUL01_Nico_Hulkenberg/nichul01.png.transform/1col/image.png', 'CountryCode': 'GER'}, '24': {'RacingNumber': '24', 'BroadcastName': 'G ZHOU', 'FullName': 'ZHOU Guanyu', 'Tla': 'ZHO', 'Line': 13, 'TeamName': 'Alfa Romeo', 'TeamColour': 'C92D4B', 'FirstName': 'Guanyu', 'LastName': 'Zhou', 'Reference': 'GUAZHO01', 'NameFormat': 'LastNameIsPrimary', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/G/GUAZHO01_Guanyu_Zhou/guazho01.png.transform/1col/image.png', 'CountryCode': 'CHN'}, '11': {'RacingNumber': '11', 'BroadcastName': 'S PEREZ', 'FullName': 'Sergio PEREZ', 'Tla': 'PER', 'Line': 16, 'TeamName': 'Red Bull Racing', 'TeamColour': '3671C6', 'FirstName': 'Sergio', 'LastName': 'Perez', 'Reference': 'SERPER01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/S/SERPER01_Sergio_Perez/serper01.png.transform/1col/image.png', 'CountryCode': 'MEX'}, '_kf': True}

In [ ]:
SessionInfo = {'Meeting': {'Key': 1210, 'Name': 'Monaco Grand Prix', 'OfficialName': 'FORMULA 1 GRAND PRIX DE MONACO 2023', 'Location': 'Monaco', 'Country': {'Key': 114, 'Code': 'MON', 'Name': 'Monaco'}, 'Circuit': {'Key': 22, 'ShortName': 'Monte Carlo'}}, 'ArchiveStatus': {'Status': 'Generating'}, 'Key': 9094, 'Type': 'Race', 'Name': 'Race', 'StartDate': '2023-05-28T15:00:00', 'EndDate': '2023-05-28T17:00:00', 'GmtOffset': '02:00:00', 'Path': '2023/2023-05-28_Monaco_Grand_Prix/2023-05-28_Race/', '_kf': True}

In [ ]:
timingdata = {'Lines': {'1': {'GapToLeader': 'LAP 61', 'IntervalToPositionAhead': {'Value': 'LAP 61', 'Catching': False}, 'Line': 1, 'Position': '1', 'ShowPosition': True, 'RacingNumber': '1', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '26.984', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '26.984'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '47.156'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.228'}], 'Speeds': {'I1': {'Value': '133', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.604', 'Lap': 23}, 'LastLapTime': {'Value': '1:41.540', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 60, 'NumberOfPitStops': 1}, '14': {'GapToLeader': '+16.302', 'IntervalToPositionAhead': {'Value': '+16.302', 'Catching': True}, 'Line': 2, 'Position': '2', 'ShowPosition': True, 'RacingNumber': '14', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '26.776', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}], 'PreviousValue': '26.776'}, {'Stopped': False, 'Value': '47.743', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '47.743'}, {'Stopped': False, 'Value': '26.119', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.119'}], 'Speeds': {'I1': {'Value': '138', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '132', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '250', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '267', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.674', 'Lap': 43}, 'LastLapTime': {'Value': '1:40.638', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 60, 'NumberOfPitStops': 2}, '31': {'GapToLeader': '+29.585', 'IntervalToPositionAhead': {'Value': '+12.703', 'Catching': False}, 'Line': 3, 'Position': '3', 'ShowPosition': True, 'RacingNumber': '31', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.474', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.474'}, {'Stopped': False, 'Value': '48.208', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '48.208'}, {'Stopped': False, 'Value': '26.845', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.845'}], 'Speeds': {'I1': {'Value': '125', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '131', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '247', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '252', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.528', 'Lap': 41}, 'LastLapTime': {'Value': '1:42.527', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 60, 'NumberOfPitStops': 2}, '55': {'GapToLeader': '+46.529', 'IntervalToPositionAhead': {'Value': '+1.330', 'Catching': True}, 'Line': 8, 'Position': '8', 'ShowPosition': True, 'RacingNumber': '55', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '28.314', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '28.314'}, {'Stopped': False, 'Value': '47.757', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '47.757'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.971'}], 'Speeds': {'I1': {'Value': '125', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '133', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '269', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.449', 'Lap': 43}, 'LastLapTime': {'Value': '1:42.820', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 2}, '44': {'GapToLeader': '+31.628', 'IntervalToPositionAhead': {'Value': '+2.197', 'Catching': False}, 'Line': 4, 'Position': '4', 'ShowPosition': True, 'RacingNumber': '44', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.600', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.600'}, {'Stopped': False, 'Value': '48.261', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '48.261'}, {'Stopped': False, 'Value': '27.180', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.180'}], 'Speeds': {'I1': {'Value': '123', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '132', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '232', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '243', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:15.650', 'Lap': 33}, 'LastLapTime': {'Value': '1:43.041', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 60, 'NumberOfPitStops': 2}, '16': {'GapToLeader': '+43.396', 'IntervalToPositionAhead': {'Value': '+5.596', 'Catching': True}, 'Line': 6, 'Position': '6', 'ShowPosition': True, 'RacingNumber': '16', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.831', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.831'}, {'Stopped': False, 'Value': '48.554', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '48.554'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}], 'PreviousValue': '26.470'}], 'Speeds': {'I1': {'Value': '138', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '130', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '239', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:15.773', 'Lap': 46}, 'LastLapTime': {'Value': '1:42.305', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 2}, '10': {'GapToLeader': '+45.432', 'IntervalToPositionAhead': {'Value': '+2.036', 'Catching': False}, 'Line': 7, 'Position': '7', 'ShowPosition': True, 'RacingNumber': '10', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '28.013', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '28.013'}, {'Stopped': False, 'Value': '47.844', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '47.844'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}], 'PreviousValue': '27.764'}], 'Speeds': {'I1': {'Value': '133', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '135', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '259', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:15.831', 'Lap': 49}, 'LastLapTime': {'Value': '1:43.477', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 2}, '63': {'GapToLeader': '+37.889', 'IntervalToPositionAhead': {'Value': '+6.613', 'Catching': True}, 'Line': 5, 'Position': '5', 'ShowPosition': True, 'RacingNumber': '63', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '28.062', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '28.062'}, {'Stopped': False, 'Value': '48.313', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '48.313'}, {'Stopped': False, 'Value': '27.155', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.155'}], 'Speeds': {'I1': {'Value': '131', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '129', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '247', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '259', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.798', 'Lap': 48}, 'LastLapTime': {'Value': '1:43.530', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 60, 'NumberOfPitStops': 1}, '22': {'GapToLeader': '+90.746', 'IntervalToPositionAhead': {'Value': '+45.340', 'Catching': False}, 'Line': 9, 'Position': '9', 'ShowPosition': True, 'RacingNumber': '22', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '28.018', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '28.018'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '48.912'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.572'}], 'Speeds': {'I1': {'Value': '120', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.680', 'Lap': 36}, 'LastLapTime': {'Value': '1:42.608', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 1}, '4': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+11.084', 'Catching': True}, 'Line': 10, 'Position': '10', 'ShowPosition': True, 'RacingNumber': '4', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '26.406', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '26.406'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '47.759'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.446'}], 'Speeds': {'I1': {'Value': '140', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.844', 'Lap': 46}, 'LastLapTime': {'Value': '1:41.216', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 2}, '81': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+2.029', 'Catching': False}, 'Line': 11, 'Position': '11', 'ShowPosition': True, 'RacingNumber': '81', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '26.216', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '26.216'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '48.366'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.163'}], 'Speeds': {'I1': {'Value': '132', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.513', 'Lap': 47}, 'LastLapTime': {'Value': '1:42.379', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 1}, '21': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+16.217', 'Catching': False}, 'Line': 13, 'Position': '13', 'ShowPosition': True, 'RacingNumber': '21', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '28.164', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '28.164'}, {'Stopped': False, 'Value': '49.132', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '49.132'}, {'Stopped': False, 'Value': '27.089', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.089'}], 'Speeds': {'I1': {'Value': '122', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '130', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '248', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '266', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.561', 'Lap': 43}, 'LastLapTime': {'Value': '1:44.385', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 1}, '23': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+0.960', 'Catching': True}, 'Line': 15, 'Position': '15', 'ShowPosition': True, 'RacingNumber': '23', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.907', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.907'}, {'Stopped': False, 'Value': '48.838', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '48.838'}, {'Stopped': False, 'Value': '26.388', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.014'}], 'Speeds': {'I1': {'Value': '129', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '136', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '243', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '220', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.672', 'Lap': 24}, 'LastLapTime': {'Value': '1:43.133', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 2}, '18': {'GapToLeader': '', 'IntervalToPositionAhead': {'Value': '', 'Catching': False}, 'Line': 20, 'Position': '20', 'ShowPosition': False, 'RacingNumber': '18', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': True, 'Status': 4, 'Sectors': [{'Stopped': False, 'Value': '24.037', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '24.037'}, {'Stopped': True, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2052}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '47.082'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '25.150'}], 'Speeds': {'I1': {'Value': '140', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.769', 'Lap': 18}, 'LastLapTime': {'Value': '1:36.321', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 53, 'NumberOfPitStops': 1}, '77': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+7.547', 'Catching': False}, 'Line': 12, 'Position': '12', 'ShowPosition': True, 'RacingNumber': '77', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.527', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.527'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '48.458'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.225'}], 'Speeds': {'I1': {'Value': '136', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.824', 'Lap': 19}, 'LastLapTime': {'Value': '1:42.622', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 59, 'NumberOfPitStops': 1}, '2': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+28.437', 'Catching': False}, 'Line': 18, 'Position': '18', 'ShowPosition': True, 'RacingNumber': '2', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.647', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.647'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '48.845'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.803'}], 'Speeds': {'I1': {'Value': '116', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:17.302', 'Lap': 32}, 'LastLapTime': {'Value': '1:42.599', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 58, 'NumberOfPitStops': 3}, '20': {'GapToLeader': '2 L', 'IntervalToPositionAhead': {'Value': '+80.931', 'Catching': False}, 'Line': 19, 'Position': '19', 'ShowPosition': True, 'RacingNumber': '20', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '68.435', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '68.435'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}], 'PreviousValue': '49.120'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.476'}], 'Speeds': {'I1': {'Value': '130', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '231', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:18.351', 'Lap': 41}, 'LastLapTime': {'Value': '1:48.018', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfLaps': 57, 'NumberOfPitStops': 1}, '27': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+9.858', 'Catching': True}, 'Line': 16, 'Position': '16', 'ShowPosition': True, 'RacingNumber': '27', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '28.367', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '28.367'}, {'Stopped': False, 'Value': '47.690', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '47.690'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '26.797'}], 'Speeds': {'I1': {'Value': '129', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '131', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '259', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.991', 'Lap': 6}, 'LastLapTime': {'Value': '1:42.188', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfPitStops': 2, 'NumberOfLaps': 58}, '24': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+13.676', 'Catching': True}, 'Line': 14, 'Position': '14', 'ShowPosition': True, 'RacingNumber': '24', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.562', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.562'}, {'Stopped': False, 'Value': '48.727', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '48.727'}, {'Stopped': False, 'Value': '27.303', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.303'}], 'Speeds': {'I1': {'Value': '126', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '128', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '248', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '268', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.926', 'Lap': 5}, 'LastLapTime': {'Value': '1:43.592', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfPitStops': 2, 'NumberOfLaps': 59}, '11': {'GapToLeader': '1 L', 'IntervalToPositionAhead': {'Value': '+12.346', 'Catching': True}, 'Line': 17, 'Position': '17', 'ShowPosition': True, 'RacingNumber': '11', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '27.084', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '27.084'}, {'Stopped': False, 'Value': '46.715', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '46.715'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '27.103'}], 'Speeds': {'I1': {'Value': '117', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'I2': {'Value': '132', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'FL': {'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'ST': {'Value': '264', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}}, 'BestLapTime': {'Value': '1:16.269', 'Lap': 5}, 'LastLapTime': {'Value': '1:45.989', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False}, 'NumberOfPitStops': 4, 'NumberOfLaps': 58}}, 'Withheld': False, '_kf': True}


In [ ]:
SessionData = {'Series': [{'Utc': '2023-05-28T12:01:04.945Z', 'Lap': 1}, {'Utc': '2023-05-28T13:04:28.394Z', 'Lap': 2}, {'Utc': '2023-05-28T13:05:47.783Z', 'Lap': 3}, {'Utc': '2023-05-28T13:07:06.86Z', 'Lap': 4}, {'Utc': '2023-05-28T13:08:24.983Z', 'Lap': 5}, {'Utc': '2023-05-28T13:09:42.997Z', 'Lap': 6}, {'Utc': '2023-05-28T13:11:00.652Z', 'Lap': 7}, {'Utc': '2023-05-28T13:12:18.292Z', 'Lap': 8}, {'Utc': '2023-05-28T13:13:35.665Z', 'Lap': 9}, {'Utc': '2023-05-28T13:14:52.711Z', 'Lap': 10}, {'Utc': '2023-05-28T13:16:09.787Z', 'Lap': 11}, {'Utc': '2023-05-28T13:17:26.63Z', 'Lap': 12}, {'Utc': '2023-05-28T13:18:43.644Z', 'Lap': 13}, {'Utc': '2023-05-28T13:20:01.08Z', 'Lap': 14}, {'Utc': '2023-05-28T13:21:18.157Z', 'Lap': 15}, {'Utc': '2023-05-28T13:22:35.249Z', 'Lap': 16}, {'Utc': '2023-05-28T13:23:52.092Z', 'Lap': 17}, {'Utc': '2023-05-28T13:25:08.903Z', 'Lap': 18}, {'Utc': '2023-05-28T13:26:25.651Z', 'Lap': 19}, {'Utc': '2023-05-28T13:27:42.431Z', 'Lap': 20}, {'Utc': '2023-05-28T13:28:59.68Z', 'Lap': 21}, {'Utc': '2023-05-28T13:30:16.476Z', 'Lap': 22}, {'Utc': '2023-05-28T13:31:33.13Z', 'Lap': 23}, {'Utc': '2023-05-28T13:32:49.754Z', 'Lap': 24}, {'Utc': '2023-05-28T13:34:06.518Z', 'Lap': 25}, {'Utc': '2023-05-28T13:35:23.704Z', 'Lap': 26}, {'Utc': '2023-05-28T13:36:40.922Z', 'Lap': 27}, {'Utc': '2023-05-28T13:37:58.186Z', 'Lap': 28}, {'Utc': '2023-05-28T13:39:15.95Z', 'Lap': 29}, {'Utc': '2023-05-28T13:40:33.98Z', 'Lap': 30}, {'Utc': '2023-05-28T13:41:52.135Z', 'Lap': 31}, {'Utc': '2023-05-28T13:43:10.899Z', 'Lap': 32}, {'Utc': '2023-05-28T13:44:29.679Z', 'Lap': 33}, {'Utc': '2023-05-28T13:45:48.631Z', 'Lap': 34}, {'Utc': '2023-05-28T13:47:06.676Z', 'Lap': 35}, {'Utc': '2023-05-28T13:48:24.675Z', 'Lap': 36}, {'Utc': '2023-05-28T13:49:42.658Z', 'Lap': 37}, {'Utc': '2023-05-28T13:50:59.969Z', 'Lap': 38}, {'Utc': '2023-05-28T13:52:17.077Z', 'Lap': 39}, {'Utc': '2023-05-28T13:53:34.044Z', 'Lap': 40}, {'Utc': '2023-05-28T13:54:51.199Z', 'Lap': 41}, {'Utc': '2023-05-28T13:56:08.167Z', 'Lap': 42}, {'Utc': '2023-05-28T13:57:24.946Z', 'Lap': 43}, {'Utc': '2023-05-28T13:58:41.883Z', 'Lap': 44}, {'Utc': '2023-05-28T13:59:58.616Z', 'Lap': 45}, {'Utc': '2023-05-28T14:01:15.333Z', 'Lap': 46}, {'Utc': '2023-05-28T14:02:32.394Z', 'Lap': 47}, {'Utc': '2023-05-28T14:03:49.565Z', 'Lap': 48}, {'Utc': '2023-05-28T14:05:06.641Z', 'Lap': 49}, {'Utc': '2023-05-28T14:06:24.031Z', 'Lap': 50}, {'Utc': '2023-05-28T14:07:41.357Z', 'Lap': 51}, {'Utc': '2023-05-28T14:08:58.778Z', 'Lap': 52}, {'Utc': '2023-05-28T14:10:22.839Z', 'Lap': 53}, {'Utc': '2023-05-28T14:11:52.759Z', 'Lap': 54}, {'Utc': '2023-05-28T14:13:31.679Z', 'Lap': 55}, {'Utc': '2023-05-28T14:15:42.302Z', 'Lap': 56}, {'Utc': '2023-05-28T14:17:28.378Z', 'Lap': 57}, {'Utc': '2023-05-28T14:19:11.86Z', 'Lap': 58}, {'Utc': '2023-05-28T14:20:54.655Z', 'Lap': 59}, {'Utc': '2023-05-28T14:22:38.482Z', 'Lap': 60}, {'Utc': '2023-05-28T14:24:19.98Z', 'Lap': 61}, {'Utc': '2023-05-28T14:26:00.775Z', 'Lap': 62}, {'Utc': '2023-05-28T14:27:40.054Z', 'Lap': 63}, {'Utc': '2023-05-28T14:29:19.693Z', 'Lap': 64}, {'Utc': '2023-05-28T14:30:58.004Z', 'Lap': 65}, {'Utc': '2023-05-28T14:32:35.19Z', 'Lap': 66}, {'Utc': '2023-05-28T14:34:09.219Z', 'Lap': 67}, {'Utc': '2023-05-28T14:35:42.233Z', 'Lap': 68}, {'Utc': '2023-05-28T14:37:15.794Z', 'Lap': 69}, {'Utc': '2023-05-28T14:38:47.417Z', 'Lap': 70}, {'Utc': '2023-05-28T14:40:18.65Z', 'Lap': 71}, {'Utc': '2023-05-28T14:41:49.055Z', 'Lap': 72}, {'Utc': '2023-05-28T14:43:19.413Z', 'Lap': 73}, {'Utc': '2023-05-28T14:44:47.895Z', 'Lap': 74}, {'Utc': '2023-05-28T14:46:14.722Z', 'Lap': 75}, {'Utc': '2023-05-28T14:47:40.158Z', 'Lap': 76}, {'Utc': '2023-05-28T14:49:05.484Z', 'Lap': 77}, {'Utc': '2023-05-28T14:50:31.046Z', 'Lap': 78}], 'StatusSeries': [{'Utc': '2023-05-28T11:50:23.066Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T12:07:39.579Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T12:07:53.235Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T12:20:52.862Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T12:23:07.375Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T13:03:03.926Z', 'SessionStatus': 'Started'}, {'Utc': '2023-05-28T13:03:42.379Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T13:03:44.848Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T13:17:54.02Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T13:18:51.222Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T13:21:10.329Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T13:21:20.657Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T14:14:44.662Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T14:14:51.975Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T14:14:53.334Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T14:15:28.646Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T14:15:35.083Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T14:16:47.441Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T14:16:50.613Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T14:16:55.347Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T14:20:09.438Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T14:20:25.531Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T14:21:10.343Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T14:21:23.561Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T14:23:11.95Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T14:23:49.84Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T14:25:32.119Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T14:25:44.916Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T14:38:01.84Z', 'TrackStatus': 'Yellow'}, {'Utc': '2023-05-28T14:38:32.84Z', 'TrackStatus': 'AllClear'}, {'Utc': '2023-05-28T14:51:56.106Z', 'SessionStatus': 'Finished'}], '_kf': True}

In [ ]:
Position = {'Position': [{'Timestamp': '2023-05-28T14:57:09.8338007Z', 'Entries': {'1': {'Status': 'OnTrack', 'X': -7621, 'Y': -7141, 'Z': 503}, '2': {'Status': 'OnTrack', 'X': -6470, 'Y': -9370, 'Z': 513}, '4': {'Status': 'OnTrack', 'X': -6739, 'Y': -8965, 'Z': 512}, '10': {'Status': 'OnTrack', 'X': -6480, 'Y': -9316, 'Z': 512}, '11': {'Status': 'OnTrack', 'X': -6509, 'Y': -9262, 'Z': 517}, '14': {'Status': 'OnTrack', 'X': -7622, 'Y': -7130, 'Z': 503}, '16': {'Status': 'OnTrack', 'X': -6479, 'Y': -9321, 'Z': 512}, '18': {'Status': 'OnTrack', 'X': -213, 'Y': -930, 'Z': 544}, '20': {'Status': 'OnTrack', 'X': -7184, 'Y': -7748, 'Z': 511}, '21': {'Status': 'OnTrack', 'X': -6704, 'Y': -9018, 'Z': 513}, '22': {'Status': 'OnTrack', 'X': -6656, 'Y': -9088, 'Z': 513}, '23': {'Status': 'OnTrack', 'X': -6701, 'Y': -9716, 'Z': 523}, '24': {'Status': 'OnTrack', 'X': -6469, 'Y': -9383, 'Z': 516}, '27': {'Status': 'OnTrack', 'X': -6591, 'Y': -9171, 'Z': 512}, '31': {'Status': 'OnTrack', 'X': -7621, 'Y': -7141, 'Z': 503}, '44': {'Status': 'OnTrack', 'X': -6459, 'Y': -9509, 'Z': 516}, '55': {'Status': 'OnTrack', 'X': -6520, 'Y': -9250, 'Z': 515}, '63': {'Status': 'OnTrack', 'X': -6735, 'Y': -8970, 'Z': 513}, '77': {'Status': 'OnTrack', 'X': -6631, 'Y': -9120, 'Z': 513}, '81': {'Status': 'OnTrack', 'X': -6584, 'Y': -9178, 'Z': 516}}}, {'Timestamp': '2023-05-28T14:57:10.0737642Z', 'Entries': {'1': {'Status': 'OnTrack', 'X': -7621, 'Y': -7141, 'Z': 503}, '2': {'Status': 'OnTrack', 'X': -6470, 'Y': -9370, 'Z': 513}, '4': {'Status': 'OnTrack', 'X': -6739, 'Y': -8965, 'Z': 512}, '10': {'Status': 'OnTrack', 'X': -6480, 'Y': -9316, 'Z': 512}, '11': {'Status': 'OnTrack', 'X': -6509, 'Y': -9262, 'Z': 517}, '14': {'Status': 'OnTrack', 'X': -7622, 'Y': -7130, 'Z': 503}, '16': {'Status': 'OnTrack', 'X': -6479, 'Y': -9321, 'Z': 512}, '18': {'Status': 'OnTrack', 'X': -213, 'Y': -930, 'Z': 544}, '20': {'Status': 'OnTrack', 'X': -7184, 'Y': -7748, 'Z': 511}, '21': {'Status': 'OnTrack', 'X': -6704, 'Y': -9018, 'Z': 513}, '22': {'Status': 'OnTrack', 'X': -6656, 'Y': -9088, 'Z': 513}, '23': {'Status': 'OnTrack', 'X': -6701, 'Y': -9716, 'Z': 523}, '24': {'Status': 'OnTrack', 'X': -6469, 'Y': -9383, 'Z': 516}, '27': {'Status': 'OnTrack', 'X': -6591, 'Y': -9171, 'Z': 512}, '31': {'Status': 'OnTrack', 'X': -7621, 'Y': -7141, 'Z': 503}, '44': {'Status': 'OnTrack', 'X': -6459, 'Y': -9509, 'Z': 516}, '55': {'Status': 'OnTrack', 'X': -6520, 'Y': -9250, 'Z': 515}, '63': {'Status': 'OnTrack', 'X': -6735, 'Y': -8970, 'Z': 513}, '77': {'Status': 'OnTrack', 'X': -6631, 'Y': -9120, 'Z': 513}, '81': {'Status': 'OnTrack', 'X': -6584, 'Y': -9178, 'Z': 516}}}]}

In [ ]:
for key in timingdata['Lines']:
    print({key: timingdata['Lines'][key]})

{'1': {'GapToLeader': 'LAP 61', 'IntervalToPositionAhead': {'Value': 'LAP 61', 'Catching': False}, 'Line': 1, 'Position': '1', 'ShowPosition': True, 'RacingNumber': '1', 'Retired': False, 'InPit': False, 'PitOut': False, 'Stopped': False, 'Status': 64, 'Sectors': [{'Stopped': False, 'Value': '26.984', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}, {'Status': 2048}], 'PreviousValue': '26.984'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 2048}, {'Status': 2048}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'PreviousValue': '47.156'}, {'Stopped': False, 'Value': '', 'Status': 0, 'OverallFastest': False, 'PersonalFastest': False, 'Segments': [{'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}, {'Status': 0}], 'Pre

In [ ]:
driverlist

{'1': {'RacingNumber': '1',
  'BroadcastName': 'M VERSTAPPEN',
  'FullName': 'Max VERSTAPPEN',
  'Tla': 'VER',
  'Line': 1,
  'TeamName': 'Red Bull Racing',
  'TeamColour': '3671C6',
  'FirstName': 'Max',
  'LastName': 'Verstappen',
  'Reference': 'MAXVER01',
  'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/M/MAXVER01_Max_Verstappen/maxver01.png.transform/1col/image.png',
  'CountryCode': 'NED'},
 '14': {'RacingNumber': '14',
  'BroadcastName': 'F ALONSO',
  'FullName': 'Fernando ALONSO',
  'Tla': 'ALO',
  'Line': 2,
  'TeamName': 'Aston Martin',
  'TeamColour': '358C75',
  'FirstName': 'Fernando',
  'LastName': 'Alonso',
  'Reference': 'FERALO01',
  'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/F/FERALO01_Fernando_Alonso/feralo01.png.transform/1col/image.png',
  'CountryCode': 'ESP'},
 '31': {'RacingNumber': '31',
  'BroadcastName': 'E OCON',
  'FullName': 'Esteban OCON',
  'Tla': 'OCO',
  'Line': 3,
  'TeamName': 'Alpine',
  'TeamC

In [ ]:
for driver in driverlist.keys():
    if driver != '_kf':
        print(driverlist[driver])

{'RacingNumber': '1', 'BroadcastName': 'M VERSTAPPEN', 'FullName': 'Max VERSTAPPEN', 'Tla': 'VER', 'Line': 1, 'TeamName': 'Red Bull Racing', 'TeamColour': '3671C6', 'FirstName': 'Max', 'LastName': 'Verstappen', 'Reference': 'MAXVER01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/M/MAXVER01_Max_Verstappen/maxver01.png.transform/1col/image.png', 'CountryCode': 'NED'}
{'RacingNumber': '14', 'BroadcastName': 'F ALONSO', 'FullName': 'Fernando ALONSO', 'Tla': 'ALO', 'Line': 2, 'TeamName': 'Aston Martin', 'TeamColour': '358C75', 'FirstName': 'Fernando', 'LastName': 'Alonso', 'Reference': 'FERALO01', 'HeadshotUrl': 'https://www.formula1.com/content/dam/fom-website/drivers/F/FERALO01_Fernando_Alonso/feralo01.png.transform/1col/image.png', 'CountryCode': 'ESP'}
{'RacingNumber': '31', 'BroadcastName': 'E OCON', 'FullName': 'Esteban OCON', 'Tla': 'OCO', 'Line': 3, 'TeamName': 'Alpine', 'TeamColour': '2293D1', 'FirstName': 'Esteban', 'LastName': 'Ocon', 'Reference': 'ES